In [2]:
import json
import os
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
#import skbox.connectors.teradata as teradata
from skboxwin import Teradata #A REMPLACER PAR SKBOX
import getpass

os.environ['TERADATA_UID'] = 'U_PROD_CHG_BIGDATA_001'
os.environ['TERADATA_PWD'] = getpass.getpass()

teradata = Teradata()


 ··········


Teradata Database ODBC Driver 16.20


In [23]:
# function to convert json file to dataframe
def json2df(file):
    tweets = []
    for line in open(file, encoding="utf8"):
        tweets.append(json.loads(line))

    df = pd.DataFrame(tweets)
    return df

alljsonsites_filename = 'scraping_websites_parse_workers_webites_results.jl'
dirpath = os.getcwd()
data_foldername = dirpath+ "\data"
alljsonsites_filename_path = data_foldername+"\\"+alljsonsites_filename 

df = json2df(alljsonsites_filename_path)
#get first value of base_url column to build netloc column
df['netloc'] = df['base_url'].apply(lambda x: x[0])

df[df.isnull()]=''
cols_txt = ['website_description', 'website_keywords', 'website_tag_div',
       'website_tag_p', 'website_tag_span', 'website_titles']
df['text_NLP'] = df[cols_txt].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

print(df.tail(2))

     base_scheme                        base_url  \
5807      [http]  [www.guignery-fils-lehavre.fr]   
5808      [http]  [www.guignery-fils-lehavre.fr]   

                                             image_urls  \
5807  [http://www.guignery-fils-lehavre.fr/pluginApp...   
5808  [http://www.guignery-fils-lehavre.fr/gallery/A...   

                                                 images lib_arty siret  \
5807  [{'url': 'http://www.guignery-fils-lehavre.fr/...                  
5808  [{'url': 'http://www.guignery-fils-lehavre.fr/...                  

                                                    url  \
5807  [http://www.guignery-fils-lehavre.fr/salles-de...   
5808  [http://www.guignery-fils-lehavre.fr/galerie-p...   

                                    website_description website_email  \
5807  [christian renov plombier salle de bains insta...            []   
5808                                                 []            []   

                                       websi

In [24]:
# get DF train = websites from ARTY
artymatching = 'data_matching_test_v2.xlsx'

df_match = pd.read_excel(data_foldername+"\\"+artymatching)
print(df_match.tail(2))

              siret                     netloc              LIB_ARTY
374  84928014400015       www.conforteteau.com        EAU ET CONFORT
375  84974738100010  www.renovation-maillot.fr  RENOV'INTERIEUR SARL


In [25]:
# get dom from ARTY with SIRET number
list_siret = list(df_match['siret'])
list_siret = [str(x) for x in list_siret]

In [26]:
#build sql to get dom from arty
sql = """SELECT art.NUM_ARTY, art.COD_SRT, art_dom.NUM_DOM, dom.lib_dom AS lib_dom
FROM BB_PROD_001_UNVBU.TD_SRV_ARTY AS art
    LEFT JOIN BB_PROD_001_UNVBU.TD_SRV_ARTY_DOM AS art_dom ON (art.num_arty= art_dom.num_arty)
    LEFT JOIN BB_PROD_001_UNVBU.TD_SRV_DOM_POS AS dom ON (art_dom.NUM_DOM = dom.NUM_DOM)
	where art_dom.num_dom is not null
    and art.DAT_DEREF_FIC is not null
	and art.COD_SRT in (""" + str(list_siret)[1:-1] + """)
	order by 2
"""

df_arty = teradata.select(sql)


In [27]:
df_arty = pd.concat(list(df_arty))
print(df_arty.tail(20))

    NUM_ARTY         COD_SRT  NUM_DOM                            lib_dom
68    923256  81224239400011     8540          R03-plomberie - sanitaire
69    923256  81224239400011    16450                   R07-SDB complete
70    923256  81224239400011     8530         R08-Chaudiere Eligible RGE
71    923256  81224239400011     7020                R07-Sanitaire lourd
72    923256  81224239400011     8120                          R08-poêle
73    923256  81224239400011    16906       R08-Chaudières non éligibles
74    923256  81224239400011     7410                  R07-Meuble de SDB
75    923256  81224239400011     8560                        R07-Robinet
76    923256  81224239400011     7420                             R07-WC
77    923951  83786775300019    16901             R01- Platrerie Cloison
78    923951  83786775300019     6350  R06-Carrelage intérieur - Faïence
79    923951  83786775300019    16933          R06 - Carrelage extérieur
80    923951  83786775300019    16925              

In [28]:
# group all domaine for each workers
s_arty = df_arty.groupby('COD_SRT')['lib_dom'].apply(lambda x: "{%s}" % ', '.join(x))
d = {'COD_SRT':s_arty.index.tolist(), 'lib_dom':s_arty.values.tolist()}
df_arty_clean = pd.DataFrame(data=d)
df_arty_clean['COD_SRT']=pd.to_numeric(df_arty_clean['COD_SRT'])
print(df_arty_clean.tail(5))


           COD_SRT                                            lib_dom
8   79804742900013  {R02-Menuiserie intérieure, R09-Portail, R02-s...
9   80250226000010  {R07-WC, R07-Robinet, R06-Parquet collé, R06-C...
10  80491224400014  {R02-Porte de garage, R09-Abri de jardin, R09-...
11  81224239400011  {R06-Carrelage intérieur - Faïence, R08-Instal...
12  83786775300019  {R01- Platrerie Cloison, R06-Carrelage intérie...


In [29]:
# get distinct lib_dom

# get DF train = websites from ARTY
artymatching = 'data_matching_test_v2.xlsx'

df_match = pd.read_excel(data_foldername+"\\"+artymatching)
print(df_match.tail(2))

              siret                     netloc              LIB_ARTY
374  84928014400015       www.conforteteau.com        EAU ET CONFORT
375  84974738100010  www.renovation-maillot.fr  RENOV'INTERIEUR SARL


In [30]:
# join arty & match
df_match = df_match.set_index('siret').join(df_arty_clean.set_index('COD_SRT'))

print(df_match.tail(5))

                                     netloc                   LIB_ARTY lib_dom
siret                                                                         
84526305200019   www.cazeaux-electricite.fr  AE ELECTRICITE RENOVATION     NaN
84749781500013      www.montage-cuisines.ch        BS MONTAGE CUISINES     NaN
84760654800018  www.styleambianceconcept.fr    GELIN CONCEPT ET STYLES     NaN
84928014400015         www.conforteteau.com             EAU ET CONFORT     NaN
84974738100010    www.renovation-maillot.fr       RENOV'INTERIEUR SARL     NaN


In [31]:
# find the arty in the json file
#/!\ bug in the json file : siret is not correctly field => get the first baseurl to compare with the netloc matching
df_match.rename(columns={"siret": "siret_match"})
df2 = df.set_index('netloc').join(df_match.set_index('netloc'),lsuffix='_left', rsuffix='_right')
subset = df2.loc[:,('base_url','LIB_ARTY','lib_dom')]
print(subset.tail(2))

                              base_url          LIB_ARTY lib_dom
netloc                                                          
zestedesavoir.com  [zestedesavoir.com]  UN ZESTE DE BOIS     NaN
zestedesavoir.com  [zestedesavoir.com]  UN ZESTE DE BOIS     NaN


In [32]:
# let pickle dataframe
pkl_df ='./SCRAP_NLP_DATAFRAME.pkl'
df2.to_pickle(pkl_df)
print("pickle est prêt !")

pickle est prêt !
